### Rating in game:
https://www.pbs.org/kcts/videogamerevolution/impact/esrb.html#:~:text=E%20%3D%20Everyone,mischief%20and%2For%20mild%20language.

### User-Agent option:
https://www.zenrows.com/blog/python-requests-user-agent#random-user-agent

In [21]:
import sys
!{sys.executable} -m pip install beautifulsoup4

In [56]:
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd
from time import sleep
from random import randint
import re

In [5]:
print("Beautiful Soup version: {}".format(bs4.__version__))
print("Request version: {}".format(requests.__version__))
print("Pandas version: {}".format(pd.__version__))

Beautiful Soup version: 4.10.0
Request version: 2.26.0
Pandas version: 1.3.4


In [ ]:
print('Python version: ')
print(sys.executable)
print(sys.version)
print(sys.version_info)

In [ ]:
print('Jupyter version: ')
!jupyter --version

In [ ]:
print('Anaconda version: ')
!conda --version

=============================================================================================

In [68]:
df = pd.read_csv('metacritic_action_fix.csv')

In [69]:
df = df.iloc[:, 1:]

In [70]:
df.head()

,id,url,image,title,platform,release_date,summary,metascore,userscore,genre
0,160633,https://www.metacritic.com/game/nintendo-64/bl...,https://static.metacritic.com/images/products/...,Blues Brothers 2000,Nintendo 64,"November 16, 2000",The newly freed Elwood scours the Windy City t...,32,3.7,Action
1,101214,https://www.metacritic.com/game/pc/star-trek-d...,https://static.metacritic.com/images/products/...,Star Trek: Deep Space Nine: The Fallen,PC,"November 15, 2000",The Federation believes that whoever controls ...,81,7.8,Action
2,161943,https://www.metacritic.com/game/dreamcast/disn...,https://static.metacritic.com/images/products/...,Disney's 102 Dalmatians: Puppies to the Rescue,Dreamcast,"November 15, 2000",It's puppy power to the rescue! As Oddball or ...,66,7.9,Action
3,101404,https://www.metacritic.com/game/pc/chessmaster...,https://static.metacritic.com/images/products/...,Chessmaster 8000,PC,"November 14, 2000",While it does have some new features and still...,80,6.7,Action
4,161948,https://www.metacritic.com/game/dreamcast/cann...,https://static.metacritic.com/images/products/...,Cannon Spike,Dreamcast,"November 14, 2000","Far into the future, a rogue group of terroris...",73,8.1,Action


In [71]:
print(df['url'][0])

https://www.metacritic.com/game/nintendo-64/blues-brothers-2000


In [72]:
df['url'] = df['url'].apply(lambda x: x+'/details')

In [73]:
print(df['url'][0])

https://www.metacritic.com/game/nintendo-64/blues-brothers-2000/details


In [99]:
id_game_list = []
pub_list = []
pub_url_list = []
rating_list = []
dev_list = []
multigenre_list = []
player_list = []

meta_how_list = []
meta_rev_list = []

user_how_list = []
user_rev_list = []

In [104]:
data = []

In [ ]:
exception_row = []

In [ ]:
# idx = 0
for index, row in df.iterrows():
#     if index < 1410:
#         continue
    print("============== Current row: {} ==============".format(index))
#     idx += 1
    
    try:
        page = requests.get(row['url'], headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(page.text, 'html.parser')
        sleep(randint(2, 4))

        # id_game = soup.find('div', {"class": "sharing"}).get('data-mc-refid') # commented
        id_game = row['id']
        publisher = soup.find('span', {"class": "data"}).find_all('a')
        pub = [x.get_text().strip() for x in publisher]
        publisher = ", ".join(p for p in pub)

        publisher_url = 'https://www.metacritic.com' + soup.find('span', {"class": "data"}).find('a').get('href')

        container = soup.find_all('div', {"class": "product_details"})[1].find_all('tr')

        rating = ""
        developer = ""
        multi_genre = ""
        player = ""
        for con in container:
            title = con.find('th').get_text()
            if(title == 'Rating:'):
                if(rating == ""):
                    rating = con.find('td').get_text()
            elif(title == 'Developer:'):
                if(developer == ""):
                    developer = con.find('td').get_text()
            elif(title == 'Genre(s):'):
                if(multi_genre == ""):
                    multi_genre = con.find('td').get_text().strip()
                    multi_genre = " ".join(multi_genre.split())
            elif(title == 'Number of Online Players:'):
                if(player == ""):
                    player = con.find('td').get_text()

        metascore_how = soup.find('div', {"class": "feature_metascore"}).find('span', {"class": "desc"}).get_text().strip()
        metascore_review = soup.find('div', {"class": "feature_metascore"}).find('span', {"class": "count"}).find('a').find('span').get_text().strip()

        userscore_how = soup.find('div', {"class": "feature_userscore"}).find('span', {"class": "desc"}).get_text().strip()
        userscore_review = soup.find('div', {"class": "feature_userscore"}).find('span', {"class": "count"}).find('a')

        u_review = ""
        if (userscore_review != None):
            u_review = userscore_review.get_text()
            u_review = re.findall(r'\d+', u_review)[0]

        print(row['title']) # nama game
        print(row['url']) # url game
        print(id_game)
        print(publisher)
        print(publisher_url)
        print(rating)
        print(developer)
        print(multi_genre)
        print(player)
        print(metascore_how)
        print(metascore_review)
        print(userscore_how)
        print(u_review)
        
        temp_data = {
            'id': id_game,
            'publisher': publisher,
            'publisher_url': publisher_url, 
            'rating': rating,
            'developer': developer,
            'multigenre': multi_genre,
            'player': player,
            'metascore_how': metascore_how,
            'metascore_review': metascore_review,
            'userscore_how': userscore_how,
            'userscore_review': u_review
        }
        data.append(temp_data)
        
    except Exception as e:
        exception_row.append(index)
        df2 = pd.DataFrame(data)
        df2.to_csv('metacritic_action_DETAILED_fix.csv')
        print(df2.duplicated(subset=['id', 'publisher', 'publisher_url', 'rating', 'developer', 'multigenre', 'player', 'metascore_how', 'metascore_review', 'userscore_how', 'userscore_review']).unique())
        print(len(df2['id'].unique()))
        print("Exception!!!")
        print(e)
        
        with open('action_DETAIL_exception.txt', 'w') as f:
            for i in exception_row:
                f.write(i)
                f.write("\n")

============== Current row: 0 ==============
Blues Brothers 2000
https://www.metacritic.com/game/nintendo-64/blues-brothers-2000/details
160633
Titus Software
https://www.metacritic.com/company/titus-software
E
Player 1
Action, Platformer, 3D

Generally unfavorable reviews
6
Generally unfavorable reviews
13
============== Current row: 1 ==============
Star Trek: Deep Space Nine: The Fallen
https://www.metacritic.com/game/pc/star-trek-deep-space-nine-the-fallen/details
101214
Simon & Schuster
https://www.metacritic.com/company/simon-schuster
T
Double Helix Games
Action, Shooter, Shooter, Third-Person, Sci-Fi, Arcade

Generally favorable reviews
23
Generally favorable reviews
25
============== Current row: 2 ==============
Disney's 102 Dalmatians: Puppies to the Rescue
https://www.metacritic.com/game/dreamcast/disneys-102-dalmatians-puppies-to-the-rescue/details
161943
Eidos Interactive
https://www.metacritic.com/company/eidos-interactive
E
Crystal Dynamics
Action, Platformer, 3D

Mixed 

Tekken Tag Tournament
https://www.metacritic.com/game/playstation-2/tekken-tag-tournament/details
101153
Namco
https://www.metacritic.com/company/namco
T
Namco
Action, Fighting, 3D

Generally favorable reviews
25
Generally favorable reviews
181
============== Current row: 28 ==============
DOA2: Hardcore
https://www.metacritic.com/game/playstation-2/doa2-hardcore/details
100468
Tecmo
https://www.metacritic.com/company/tecmo
T
Team Ninja
Action, Fighting, 3D

Universal acclaim
20
Generally favorable reviews
70
============== Current row: 29 ==============
Dynasty Warriors 2
https://www.metacritic.com/game/playstation-2/dynasty-warriors-2/details
100508
Koei
https://www.metacritic.com/company/koei
T
Omega Force
Action, Beat-'Em-Up

Generally favorable reviews
17
Generally favorable reviews
40
============== Current row: 30 ==============
Deep Raider
https://www.metacritic.com/game/pc/deep-raider/details
101389
InfoBank
https://www.metacritic.com/company/infobank
T
Cyberscape Graphics
Act

Army Men: Air Attack 2
https://www.metacritic.com/game/playstation/army-men-air-attack-2/details
160821
3DO
https://www.metacritic.com/company/3do
T
3DO
Action, General

Mixed or average reviews
8
Mixed or average reviews
10
============== Current row: 54 ==============
Army Men: Sarge's Heroes 2
https://www.metacritic.com/game/nintendo-64/army-men-sarges-heroes-2/details
160849
3DO
https://www.metacritic.com/company/3do
T
3DO
Action, Shooter, Third-Person, Fantasy

Generally unfavorable reviews
8
Mixed or average reviews
18
============== Current row: 55 ==============
Buzz Lightyear of Star Command
https://www.metacritic.com/game/dreamcast/buzz-lightyear-of-star-command/details
160867
Activision
https://www.metacritic.com/company/activision
E
Traveller's Tales
Action, Platformer, 3D

Mixed or average reviews
16
Generally favorable reviews
6
============== Current row: 56 ==============
Buzz Lightyear of Star Command
https://www.metacritic.com/game/playstation/buzz-lightyear-of-star-c

Toy Story 2: Buzz Lightyear to the Rescue!
https://www.metacritic.com/game/dreamcast/toy-story-2-buzz-lightyear-to-the-rescue!/details
160844
Activision
https://www.metacritic.com/company/activision
E
Traveller's Tales
Action, General

Mixed or average reviews
16
Mixed or average reviews
12
============== Current row: 81 ==============
Mortal Kombat: Special Forces
https://www.metacritic.com/game/playstation/mortal-kombat-special-forces/details
160705
Midway
https://www.metacritic.com/company/midway
M
Midway
Action, Beat-'Em-Up

Generally unfavorable reviews
7
Generally unfavorable reviews
38
============== Current row: 82 ==============
Marvel vs. Capcom 2
https://www.metacritic.com/game/dreamcast/marvel-vs-capcom-2/details
161988
Capcom
https://www.metacritic.com/company/capcom
T
Capcom
Action, Fighting, 2D

Universal acclaim
22
Generally favorable reviews
101
============== Current row: 83 ==============
Kirby 64: The Crystal Shards
https://www.metacritic.com/game/nintendo-64/kirby-

Tweety & The Magic Gems
https://www.metacritic.com/game/game-boy-advance/tweety-the-magic-gems/details
163136
Kemco
https://www.metacritic.com/company/kemco
E
Kemco
Action, General

Generally unfavorable reviews
4
Generally favorable reviews
5
============== Current row: 108 ==============
Hot Potato!
https://www.metacritic.com/game/game-boy-advance/hot-potato!/details
163144
Bam Entertainment
https://www.metacritic.com/company/bam-entertainment
E
Pukka Games
Action, General

Mixed or average reviews
7
Generally favorable reviews
5
============== Current row: 109 ==============
WarJetz
https://www.metacritic.com/game/playstation-2/warjetz/details
101381
3DO
https://www.metacritic.com/company/3do
T
3DO
Action, Shooter, Third-Person, Sci-Fi

Mixed or average reviews
9
No user score yet

============== Current row: 110 ==============
Bloody Roar 3
https://www.metacritic.com/game/playstation-2/bloody-roar-3/details
101460
Activision
https://www.metacritic.com/company/activision
T
Eighting/

...

Draw Slasher
https://www.metacritic.com/game/playstation-vita/draw-slasher/details
240898
Mass Creation
https://www.metacritic.com/company/mass-creation
T
Mass Creation
Action, General, General, Beat-'Em-Up, 2D

Mixed or average reviews
6
Mixed or average reviews
14
============== Current row: 5573 ==============
Dead Island: Riptide
https://www.metacritic.com/game/xbox-360/dead-island-riptide/details
213537
Deep Silver
https://www.metacritic.com/company/deep-silver
M
Techland
Action, Shooter, Shooter, First-Person, Modern, Modern, Arcade
Up to 4 Players
Mixed or average reviews
43
Mixed or average reviews
209
============== Current row: 5574 ==============
Dead Island: Riptide
https://www.metacritic.com/game/playstation-3/dead-island-riptide/details
213538
Deep Silver
https://www.metacritic.com/company/deep-silver
M
Techland
Action, Shooter, Shooter, First-Person, Modern, Modern, Arcade
Up to 4 Players
Mixed or average reviews
25
Mixed or average reviews
160
============== Current row

[False]
5381
Exception!!!
============== Current row: 5602 ==============
[False]
5381
Exception!!!
============== Current row: 5603 ==============
Sprinkle Islands
https://www.metacritic.com/game/ios/sprinkle-islands/details
248666
Mediocre AB
https://www.metacritic.com/company/mediocre-ab


Action, General, General

Generally favorable reviews
13
No user score yet

============== Current row: 5604 ==============
Tiny Thief
https://www.metacritic.com/game/ios/tiny-thief/details
248667
Rovio Stars Ltd.
https://www.metacritic.com/company/rovio-stars-ltd

5 Ants
Action, General, General, Platformer, 2D

Generally favorable reviews
21
Mixed or average reviews
22
============== Current row: 5605 ==============
Pacific Rim
https://www.metacritic.com/game/ios/pacific-rim/details
248669
Reliance Big Entertainment UK Private Ltd
https://www.metacritic.com/company/reliance-big-entertainment-uk-private-ltd


Action, General, General, Fighting, 3D

Generally unfavorable reviews
14
Generally unfav

The Walking Dead: 400 Days
https://www.metacritic.com/game/playstation-3/the-walking-dead-400-days/details
245757
Telltale Games
https://www.metacritic.com/company/telltale-games
M
Telltale Games
Action, Miscellaneous, Adventure, General, Compilation, Point-and-Click

Generally favorable reviews
12
Generally favorable reviews
188
============== Current row: 5628 ==============
Doodle Jump for Kinect
https://www.metacritic.com/game/xbox-360/doodle-jump-for-kinect/details
245112
D3Publisher
https://www.metacritic.com/company/d3publisher
E
Lima Sky, Smoking Gun Interactive Inc., Smoking Gun Interactive
Action, Platformer, Platformer, 2D, 2D

Generally unfavorable reviews
5
Mixed or average reviews
12
============== Current row: 5629 ==============
Colin McRae Rally
https://www.metacritic.com/game/ios/colin-mcrae-rally/details
247351
Codemasters, The Codemasters Software Company Limited
https://www.metacritic.com/company/codemasters


Action, Driving, Racing, General, Simulation, Rally / O

LEGO Legends of Chima: Laval's Journey
https://www.metacritic.com/game/playstation-vita/lego-legends-of-chima-lavals-journey/details
229610
Warner Bros. Interactive Entertainment
https://www.metacritic.com/company/warner-bros-interactive-entertainment
E10+
TT Games
Action, Action Adventure, General, General, Sandbox

Mixed or average reviews
5
Mixed or average reviews
11
============== Current row: 5653 ==============
Joe Danger 2: The Movie
https://www.metacritic.com/game/pc/joe-danger-2-the-movie/details
187909
Hello Games
https://www.metacritic.com/company/hello-games

Hello Games
Action, Racing, General, General, Arcade, Other
No Online Multiplayer
Generally favorable reviews
4
Mixed or average reviews
26
============== Current row: 5654 ==============
Deadpool
https://www.metacritic.com/game/pc/deadpool/details
236356
Activision
https://www.metacritic.com/company/activision
M
High Moon Studios
Fighting, 3D, Action, Action Adventure, Beat-'Em-Up, Shooter, Beat-'Em-Up, Linear, Third

[False]
5453
Exception!!!
============== Current row: 5688 ==============
Sparkle 2
https://www.metacritic.com/game/ios/sparkle-2/details
245210
10tons Ltd, 10tons
https://www.metacritic.com/company/10tons-ltd

10tons Entertainment, 10tons
Action, Puzzle, General, General, Matching

Generally favorable reviews
6
No user score yet

============== Current row: 5689 ==============
[False]
5454
Exception!!!
============== Current row: 5690 ==============
[False]
5454
Exception!!!
============== Current row: 5691 ==============
LIMBO
https://www.metacritic.com/game/playstation-vita/limbo/details
237597
PLAYDEAD
https://www.metacritic.com/company/playdead
T
PLAYDEAD
Action, Platformer, Platformer, 2D, 2D

Generally favorable reviews
6
Generally favorable reviews
139
============== Current row: 5692 ==============
KAVINSKY
https://www.metacritic.com/game/ios/kavinsky/details
248280
Record Makers
https://www.metacritic.com/company/record-makers

Record Makers
Action, General, General

Mixed or

In [ ]:
df2 = pd.DataFrame(data)

In [ ]:
df2

In [ ]:
df2.duplicated(subset=['id', 'publisher', 'publisher_url', 'rating', 'developer', 'multigenre', 'player', 'metascore_how', 'metascore_review', 'userscore_how', 'userscore_review']).unique()

In [ ]:
len(df2['id'].unique())

In [ ]:
df2.to_csv('metacritic_action_DETAILED_fix.csv')

In [ ]:
with open('action_DETAIL_exception.txt', 'w') as f:
    for i in exception_row:
        f.write(i)
        f.write("\n")